In [1]:
from six.moves import cPickle as pickle
import numpy as np
import json
from sklearn.neighbors import KDTree
from collections import OrderedDict
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
import scipy
import matplotlib.pyplot as plt
from nltk.tokenize.moses import MosesTokenizer
tokenizer = MosesTokenizer()


In [2]:
def get_pairwise_distances(G):
    # G = pickle.load(open('data/Glove/cocotalk_glove_v2.pkl', 'r'))
    eps = 1e-6
    print("G shape:", G.shape, len(G))
    #  G[-1] = np.mean(G)
    for i in range(len(G)):
        if not np.sum(G[i] ** 2):
            print('%d) norm(g) = 0' % i)
            G[i] = eps + G[i]
    Ds = scipy.spatial.distance.pdist(G, metric='cosine')
    Ds = scipy.spatial.distance.squareform(Ds)
    As = np.diag(Ds)
    print("(scipy) sum:", np.sum(As), "min:", np.min(Ds), np.min(As), "max:", np.max(Ds), np.max(As))
    Ds = 1 - Ds / 2# map to [0,1]
    print("Shapes:", Ds.shape, np.min(Ds), np.max(Ds), np.diag(Ds))
    return Ds


def prepare_glove(ixtow, source, dim=300):
    """
    inputs:
        ixtow : index to word dictionnary of the vocab
        source: dict of the glove vectors
    """
    missing = []
    G = np.zeros((len(ixtow), dim), dtype="float32") # 300 in case of using glove
    for i in range(len(ixtow)):
        word = ixtow[i]
        #  print "word:", word
        if word in source:
            G[i] = source[word]
            if not np.sum(G[i] ** 2):
                raise ValueError("Norm of glove embedding null token %d| word %s" % (i, word) )
        else:
            missing.append(word)
        
    # pickle.dump(G, open('data/Glove/%s' % output, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
    return G, missing  

In [ ]:
# pickle.dump(Glove, open('data/Glove/glove_dict.pkl', 'w'), protocol=pickle.HIGHEST_PROTOCOL)
# Glove = pickle.load(open('data/Glove/glove_dict_fr.pkl', 'rb'))
# Glove = pickle.load(open('data/embeddings/full_image.pkl', 'rb'), encoding='iso-8859-1')

In [16]:
#Load vocab
# data = pickle.load(open('../seq2seq/data/WMT14/fr_trg.pkl', "rb"))
itow = json.load(open('../data/coco/cocotalk.json', 'r'))['ix_to_word']
itow = {int(k): v for k,v in itow.items()}
itow[0] = 'EOS'
print('Vocabulary size:', len(itow))

Vocabulary size: 9488


In [20]:
REF = "glove_coco_d512_w10" 
glove_dict = {}
with open('../data/Glove/vectors.w10.d512.txt', 'r') as f:
    for l in f.readlines():
        l = l.strip().split()
        word = l[0]
        code = np.array(l[1:])
        glove_dict[word] = code
pickle.dump(glove_dict, open('../data/Glove/%s_dict.pkl' % REF, 'wb'))

In [21]:
print("Preparing Glove embeddings matrix")
glove_matrix, mis = prepare_glove(itow, glove_dict, dim=512)
pickle.dump(glove_matrix, open('../data/Glove/%s_matrix.pkl' % REF, "wb"))
print('missing words: ', mis)
Sim = get_pairwise_distances(glove_matrix)
Sim = np.float32(Sim)
pickle.dump(Sim, open('../data/Glove/%s_similarities.pkl' % REF, 'wb'))

Preparing Glove embeddings matrix
missing words:  ['EOS', 'UNK']
G shape: (9488, 512) 9488
0) norm(g) = 0
9487) norm(g) = 0
(scipy) sum: 0.0 min: 0.0 0.0 max: 1.81367281115 0.0
Shapes: (9488, 9488) 0.0931635944248 1.0 [ 1.  1.  1. ...,  1.  1.  1.]
